In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider, IntSlider, VBox, HBox, Button, Layout, Dropdown, interactive_output, Output
from IPython.display import display, clear_output

In [3]:
%pip install -q ipywidgets ipympl

Note: you may need to restart the kernel to use updated packages.


In [12]:
# --- Data generation (regression) ---
def make_data(n_samples=120, sep=2.5, seed=0, true_w=2.0, true_b=0.5, noise_std=1.0):
    """
    Create a simple 1D regression dataset:
      x ~ Uniform(-sep, sep)
      y = true_w * x + true_b + N(0, noise_std^2)
    'sep' controls the spread/range of x (wider = more spread).
    """
    rng = np.random.default_rng(seed)
    x = rng.uniform(-sep, sep, size=n_samples)
    y = true_w * x + true_b + rng.normal(0.0, noise_std, size=n_samples)
    return x, y

# --- Initial dataset ---
x, y = make_data(sep=2.5, seed=0)

# --- Widgets ---
w    = FloatSlider(value=1.0, min=-10, max=10, step=0.01, description="w",   layout=Layout(width='300px'), continuous_update=True)
b    = FloatSlider(value=0.0, min=-10, max=10, step=0.01, description="b",   layout=Layout(width='300px'), continuous_update=True)
sep  = FloatSlider(value=2.5, min=0.5, max=8.0, step=0.1, description="sep", layout=Layout(width='300px'), continuous_update=False)
seed = IntSlider(value=0, min=0, max=9999, step=1, description="seed",       layout=Layout(width='300px'), continuous_update=False)
regen = Button(description="Regenerate data", layout=Layout(width='160px'))

out = Output()

def draw(w_val, b_val):
    """Rebuild the figure each time to avoid backend issues."""
    with out:
        out.clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(7.5, 4.5))

        # Predictions and metric
        yhat = w_val * x + b_val
        mse = np.mean((yhat - y) ** 2)

        # Scatter points
        ax.scatter(x, y, alpha=0.85, label="data")

        # Regression line across current x-range
        x_line = np.linspace(np.min(x), np.max(x), 200)
        y_line = w_val * x_line + b_val
        ax.plot(x_line, y_line, ls="--", lw=2, label=r"model $\hat{y}=wx+b$")

        ax.set_title(f"1D Linear Regression: MSE={mse:.3f} | w={w_val:.2f}, b={b_val:.2f}")
        ax.set_xlabel("x")
        ax.set_ylabel("y")
        ax.grid(True, alpha=0.3)
        ax.legend(loc="best")
        plt.show()

def on_regen_clicked(_):
    global x, y
    x, y = make_data(sep=sep.value, seed=seed.value)
    # Redraw with current model params
    draw(w.value, b.value)

# Wire interactivity
regen.on_click(on_regen_clicked)
io = interactive_output(draw, {"w_val": w, "b_val": b})

# Layout + initial render
display(VBox([
    HBox([w, b]),
    HBox([sep, seed, regen]),
    out
]))
draw(w.value, b.value)


In [13]:
# --- Data generation ---
def make_data(n_per_class=60, sep=2.5, seed=0):
    rng = np.random.default_rng(seed)
    x0 = rng.normal(loc=-sep/2, scale=1.0, size=n_per_class)
    x1 = rng.normal(loc= sep/2, scale=1.0, size=n_per_class)
    x = np.concatenate([x0, x1])
    y = np.concatenate([np.zeros_like(x0, dtype=int), np.ones_like(x1, dtype=int)])
    y_plot = y + rng.normal(0, 0.03, size=y.shape)  # small vertical jitter for visibility
    return x, y, y_plot

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# --- Initial dataset ---
x, y, y_plot = make_data(sep=2.5, seed=0)

# --- Widgets ---
w    = FloatSlider(value=1.0, min=-10, max=10, step=0.01, description="w",   layout=Layout(width='300px'), continuous_update=True)
b    = FloatSlider(value=0.0, min=-10, max=10, step=0.01, description="b",   layout=Layout(width='300px'), continuous_update=True)
sep  = FloatSlider(value=2.5, min=0.0, max=6.0, step=0.1, description="sep", layout=Layout(width='300px'), continuous_update=False)
seed = IntSlider(value=0, min=0, max=9999, step=1, description="seed",       layout=Layout(width='300px'), continuous_update=False)
mode = Dropdown(options=[("Sigmoid + 0.5", "sigmoid"), ("Hard step", "step")],
                value="sigmoid", description="mode", layout=Layout(width='220px'))
regen = Button(description="Regenerate data", layout=Layout(width='160px'))

out = Output()

def predict_labels(x_, w_, b_, mode_):
    z = w_ * x_ + b_
    if mode_ == "sigmoid":
        return (sigmoid(z) >= 0.5).astype(int)
    else:
        return (z >= 0).astype(int)

def draw(w, b, mode):
    """Rebuild the figure each time to avoid backend issues."""
    with out:
        out.clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(7.5, 4.5))

        # Predictions/metrics
        yhat = predict_labels(x, w, b, mode)
        acc = (yhat == y).mean()

        # Scatter
        ax.scatter(x[y==0], y_plot[y==0], label="class 0", alpha=0.85)
        ax.scatter(x[y==1], y_plot[y==1], label="class 1", alpha=0.85)

        # Decision boundary x = -b/w (if defined)
        if abs(w) > 1e-8:
            xb = -b / w
            ax.axvline(xb, ls="--", lw=2, label="decision boundary x=-b/w")

        ax.set_title(f"1D Perceptron ({'sigmoid' if mode=='sigmoid' else 'step'}): "
                     f"accuracy={acc*100:.1f}% | w={w:.2f}, b={b:.2f}")
        ax.set_xlabel("feature x")
        ax.set_ylabel("class (with jitter)")
        ax.grid(True, alpha=0.3)
        ax.legend(loc="best")
        plt.show()

def on_regen_clicked(_):
    global x, y, y_plot
    x, y, y_plot = make_data(sep=sep.value, seed=seed.value)
    # Trigger a redraw with current w/b/mode
    draw(w.value, b.value, mode.value)

# Wire interactivity
regen.on_click(on_regen_clicked)
io = interactive_output(draw, {"w": w, "b": b, "mode": mode})

# Layout + initial render
display(VBox([
    HBox([w, b]),
    HBox([sep, seed, regen]),
    mode,
    out
]))
draw(w.value, b.value, mode.value)
